# Hackathon Reddit Sentiment Analysis

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import re
from bs4 import BeautifulSoup   
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS as stopwords

In [143]:
# read data
events = pd.read_csv('../data/events_reddit.csv')

In [144]:
events.head()

,author,body,created_utc,subreddit,permalink,timestamp
0,DjMagicTouch,You're thinking in extremes man. Just like eve...,1590177465,Coronavirus,/r/Coronavirus/comments/gojqi1/daily_discussio...,2020-05-22
1,lisaseileise,"No, what is so hard to understand here? It‘s j...",1590177616,Coronavirus,/r/Coronavirus/comments/gonje6/the_coronavirus...,2020-05-22
2,jeopardy987987,"it's killed nearly 100,000 in the US in just a...",1590177796,Coronavirus,/r/Coronavirus/comments/gokm4t/a_majority_of_a...,2020-05-22
3,gp_dude,You are living in an alternate reality if you ...,1590177806,Coronavirus,/r/Coronavirus/comments/goql7x/sweden_escapes_...,2020-05-22
4,gp_dude,"No, it's not accurate, that's what the officia...",1590178095,Coronavirus,/r/Coronavirus/comments/goql7x/sweden_escapes_...,2020-05-22


In [145]:
remove = ['against','enough','only','no','not']
append = ['like','don','gt','https','www','scmp','com','x200b']

In [146]:
custom_stops = list(stopwords)
for word in remove:
    custom_stops.remove(word)
for word in append:
    custom_stops.append(word)

In [147]:
stripped_comments = []

for comment in events['body']:
    letters_only = re.sub("[^a-zA-Z]"," ", comment)
    words = letters_only.lower().split()
    meaningful_words = [w for w in words if w not in custom_stops] 
    new_body = " ".join(meaningful_words)
    stripped_comments.append(new_body)

events['stripped'] = stripped_comments

In [148]:
cvec = CountVectorizer(stop_words=custom_stops, ngram_range=(2,4))
X_text = events['body']
X_text = cvec.fit_transform(X_text)
X_text_df = pd.DataFrame(X_text.toarray(), columns = cvec.get_feature_names())

In [149]:
X_text_df.sum().sort_values(ascending=False)[:50]

social distancing             136
covid 19                      124
stay home                      68
wearing masks                  49
wear mask                      43
second wave                    38
does not                       36
did not                        33
wear masks                     32
people not                     32
new york                       32
not just                       31
public health                  29
not going                      28
2020 05                        27
wearing mask                   27
herd immunity                  25
nursing homes                  25
long term                      24
distancing measures            24
not sure                       23
social media                   23
not only                       22
contact tracing                21
no longer                      21
social distancing measures     19
lot people                     18
2020 04                        18
just not                       18
not fan       

In [142]:
[text for text in events['stripped'] if 'not sure' in text]

['scares flipping channels landed qvc accident selling breathable masks masks let mr corona right lungs bosses want open office staff june m not sure office safe require makes arrangements travel safely possible pay mortgage risk bounced house situation absolutely insane',
 'muh afganistan bold assume t think huge waste money ve supported m not sure point complaining taxpayer money used m pointing non issue trillions spent waging war apparently money isn t available people safe home no evidence people want work ll stick opinion no choice point stay home not knock effects virus affect stay home father heart attack hospital t cope example realise poorer people live extended family t really distance',
 'interesting tangents not sure states lying numbers states reopening masks social distancing m rabbit search new hole did scan applicable executive order sounds event violation just thought funny care comes great michigan actually took action against president',
 'aspies thrivvvvinnnggg no 